# Professional Fraud Detection Dashboard

## Tutorial: Building Enterprise-Grade ML Dashboards

In this comprehensive tutorial, we'll create a production-ready fraud detection dashboard that serves multiple stakeholders with real-time insights and interactive analytics.

### Learning Objectives

By the end of this tutorial, you will:

1. **Understand dashboard user personas and their needs**
2. **Build multi-level dashboards for different audiences**
3. **Implement real-time transaction monitoring**
4. **Create interactive visualizations with business context**
5. **Design explainable AI interfaces**
6. **Calculate and display business impact metrics**
7. **Build model monitoring and A/B testing dashboards**

### Prerequisites

- Completion of previous fraud detection tutorials
- Understanding of Streamlit or similar frameworks
- Basic knowledge of visualization libraries
- Familiarity with business metrics

## 1. Understanding Dashboard User Personas

Professional dashboards must serve different user types with varying needs and technical expertise.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import json
import random
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')

# Define user personas
class UserPersona:
    """Define different user types and their dashboard needs."""
    
    def __init__(self, name: str, role: str, priorities: List[str], 
                 technical_level: str, key_metrics: List[str]):
        self.name = name
        self.role = role
        self.priorities = priorities
        self.technical_level = technical_level
        self.key_metrics = key_metrics

# Define our user personas
personas = [
    UserPersona(
        name="Security Analyst Sarah",
        role="Fraud Analyst",
        priorities=["Real-time alerts", "Investigation tools", "Pattern analysis"],
        technical_level="High",
        key_metrics=["Alert volume", "False positive rate", "Investigation time"]
    ),
    UserPersona(
        name="Data Scientist Dan",
        role="ML Engineer",
        priorities=["Model performance", "Feature importance", "A/B testing"],
        technical_level="Expert",
        key_metrics=["F1 score", "Precision/Recall", "Model drift"]
    ),
    UserPersona(
        name="Manager Maria",
        role="Fraud Prevention Manager",
        priorities=["Business impact", "Team efficiency", "Cost reduction"],
        technical_level="Medium",
        key_metrics=["Fraud losses prevented", "ROI", "Team productivity"]
    ),
    UserPersona(
        name="Executive Eric",
        role="Chief Risk Officer",
        priorities=["Strategic overview", "Compliance", "Revenue protection"],
        technical_level="Low",
        key_metrics=["Total fraud rate", "Customer impact", "Regulatory compliance"]
    )
]

# Display personas
print("🎭 Dashboard User Personas:\n")
for persona in personas:
    print(f"👤 {persona.name} - {persona.role}")
    print(f"   Priorities: {', '.join(persona.priorities)}")
    print(f"   Technical Level: {persona.technical_level}")
    print(f"   Key Metrics: {', '.join(persona.key_metrics)}")
    print()

## 2. Building the Data Layer

Let's create realistic data generators for our dashboard.

In [ ]:
class TransactionSimulator:
    """Simulate realistic transaction data for dashboard testing."""
    
    def __init__(self):
        self.transaction_id = 0
        self.merchants = ['Amazon', 'Walmart', 'Target', 'BestBuy', 'Apple', 
                         'Starbucks', 'McDonalds', 'Gas Station', 'ATM']
        self.countries = ['US', 'UK', 'CA', 'FR', 'DE', 'JP', 'AU', 'BR', 'MX']
        
    def generate_transaction(self) -> Dict:
        """Generate a single transaction with fraud detection results."""
        self.transaction_id += 1
        
        # Determine if fraud (5% fraud rate)
        is_actual_fraud = random.random() < 0.05
        
        # Generate transaction details
        if is_actual_fraud:
            # Fraud patterns: high amounts, unusual merchants, foreign countries
            amount = random.choice([random.uniform(500, 2000), 
                                  random.uniform(1, 10)])  # High or very low
            merchant = random.choice(self.merchants + ['Unknown Merchant'])
            country = random.choice(self.countries + ['XX', 'YY'])  # Include unusual
            risk_score = random.uniform(0.7, 0.95)
        else:
            # Normal patterns
            amount = random.uniform(10, 300)
            merchant = random.choice(self.merchants)
            country = random.choice(self.countries[:3])  # Mostly US, UK, CA
            risk_score = random.uniform(0.05, 0.4)
        
        # Model prediction (80% accurate)
        if random.random() < 0.8:
            is_predicted_fraud = risk_score > 0.5
        else:
            is_predicted_fraud = not (risk_score > 0.5)
        
        return {
            'transaction_id': f'TXN{self.transaction_id:06d}',
            'timestamp': datetime.now() - timedelta(seconds=random.randint(0, 3600)),
            'amount': round(amount, 2),
            'merchant': merchant,
            'country': country,
            'customer_id': f'CUST{random.randint(1000, 9999)}',
            'risk_score': round(risk_score, 3),
            'is_predicted_fraud': is_predicted_fraud,
            'is_actual_fraud': is_actual_fraud,
            'model_version': random.choice(['v1.2.0', 'v1.2.1', 'v1.3.0']),
            'processing_time_ms': random.uniform(5, 50)
        }
    
    def generate_batch(self, n: int = 100) -> pd.DataFrame:
        """Generate batch of transactions."""
        transactions = [self.generate_transaction() for _ in range(n)]
        return pd.DataFrame(transactions)

# Generate sample data
simulator = TransactionSimulator()
transactions_df = simulator.generate_batch(1000)

print("📊 Generated Transaction Data:")
print(f"Total transactions: {len(transactions_df)}")
print(f"Fraud rate: {transactions_df['is_actual_fraud'].mean():.2%}")
print(f"Detection rate: {transactions_df['is_predicted_fraud'].mean():.2%}")
print("\nSample transactions:")
print(transactions_df.head())

## 3. Creating the Dashboard Architecture

Let's design a multi-level dashboard architecture that serves all user personas.

In [ ]:
class DashboardArchitecture:
    """Define the overall dashboard structure and components."""
    
    def __init__(self):
        self.components = {
            'executive': [
                'KPI Summary Cards',
                'Trend Charts',
                'Geographic Heatmap',
                'Risk Overview'
            ],
            'manager': [
                'Team Performance Metrics',
                'Cost-Benefit Analysis',
                'Alert Distribution',
                'Investigation Pipeline'
            ],
            'analyst': [
                'Real-time Alert Feed',
                'Investigation Tools',
                'Pattern Analysis',
                'Case Management'
            ],
            'data_scientist': [
                'Model Performance Metrics',
                'Feature Importance',
                'A/B Test Results',
                'Drift Detection'
            ]
        }
    
    def get_layout(self, user_type: str) -> Dict:
        """Get dashboard layout for specific user type."""
        return {
            'title': f'{user_type.title()} Fraud Dashboard',
            'components': self.components.get(user_type, []),
            'refresh_rate': 60 if user_type == 'analyst' else 300,  # seconds
            'theme': 'dark' if user_type == 'analyst' else 'light'
        }

# Create dashboard architecture
architecture = DashboardArchitecture()

# Display architecture
print("🏗️ Dashboard Architecture:\n")
for user_type, components in architecture.components.items():
    layout = architecture.get_layout(user_type)
    print(f"📊 {layout['title']}")
    print(f"   Refresh: Every {layout['refresh_rate']}s")
    print(f"   Theme: {layout['theme']}")
    print("   Components:")
    for component in components:
        print(f"     - {component}")
    print()

## 4. Building Interactive Visualizations

Let's create the core visualization components for our dashboard.

In [ ]:
class DashboardVisualizer:
    """Create interactive visualizations for fraud dashboard."""
    
    def __init__(self, transactions_df: pd.DataFrame):
        self.df = transactions_df
        self.colors = {
            'primary': '#1f77b4',
            'success': '#2ca02c',
            'warning': '#ff7f0e',
            'danger': '#d62728',
            'info': '#17a2b8'
        }
    
    def create_kpi_cards(self) -> go.Figure:
        """Create KPI summary cards for executives."""
        # Calculate KPIs
        total_transactions = len(self.df)
        fraud_rate = self.df['is_actual_fraud'].mean()
        detection_accuracy = (
            (self.df['is_predicted_fraud'] == self.df['is_actual_fraud']).mean()
        )
        total_amount = self.df['amount'].sum()
        fraud_prevented = self.df[
            self.df['is_predicted_fraud'] & self.df['is_actual_fraud']
        ]['amount'].sum()
        
        # Create figure with subplots
        fig = make_subplots(
            rows=1, cols=4,
            subplot_titles=(
                'Total Transactions',
                'Fraud Rate',
                'Detection Accuracy',
                'Fraud Prevented'
            ),
            specs=[[{'type': 'indicator'}, {'type': 'indicator'}, 
                   {'type': 'indicator'}, {'type': 'indicator'}]]
        )
        
        # Add KPI indicators
        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=total_transactions,
                delta={'reference': 950, 'relative': True},
                domain={'x': [0, 0.25], 'y': [0, 1]}
            ),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=fraud_rate * 100,
                domain={'x': [0.25, 0.5], 'y': [0, 1]},
                gauge={'axis': {'range': [None, 10]},
                       'bar': {'color': self.colors['danger']},
                       'threshold': {
                           'line': {'color': "red", 'width': 4},
                           'thickness': 0.75,
                           'value': 7
                       }}
            ),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Indicator(
                mode="gauge+number",
                value=detection_accuracy * 100,
                domain={'x': [0.5, 0.75], 'y': [0, 1]},
                gauge={'axis': {'range': [None, 100]},
                       'bar': {'color': self.colors['success']},
                       'steps': [
                           {'range': [0, 50], 'color': "lightgray"},
                           {'range': [50, 80], 'color': "gray"}],
                       'threshold': {
                           'line': {'color': "green", 'width': 4},
                           'thickness': 0.75,
                           'value': 90
                       }}
            ),
            row=1, col=3
        )
        
        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=fraud_prevented,
                number={'prefix': "$", 'valueformat': ",.0f"},
                delta={'reference': fraud_prevented * 0.9, 'relative': True,
                       'valueformat': ".1%"},
                domain={'x': [0.75, 1], 'y': [0, 1]}
            ),
            row=1, col=4
        )
        
        fig.update_layout(
            title="Executive KPI Dashboard",
            height=300,
            showlegend=False
        )
        
        return fig
    
    def create_real_time_feed(self) -> go.Figure:
        """Create real-time transaction feed for analysts."""
        # Get recent high-risk transactions
        recent_alerts = self.df[
            self.df['is_predicted_fraud']
        ].nlargest(20, 'timestamp')
        
        # Create table
        fig = go.Figure(data=[go.Table(
            header=dict(
                values=['Time', 'Transaction ID', 'Amount', 'Merchant', 
                       'Risk Score', 'Status'],
                fill_color=self.colors['primary'],
                font=dict(color='white', size=12),
                align='left'
            ),
            cells=dict(
                values=[
                    recent_alerts['timestamp'].dt.strftime('%H:%M:%S'),
                    recent_alerts['transaction_id'],
                    ['$' + str(x) for x in recent_alerts['amount']],
                    recent_alerts['merchant'],
                    [f"{x:.1%}" for x in recent_alerts['risk_score']],
                    ['🚨 ALERT' if x else 'Monitoring' 
                     for x in recent_alerts['is_predicted_fraud']]
                ],
                fill_color=[
                    ['white'] * len(recent_alerts),
                    ['white'] * len(recent_alerts),
                    ['white'] * len(recent_alerts),
                    ['white'] * len(recent_alerts),
                    [self.colors['danger'] if x > 0.8 else 
                     self.colors['warning'] if x > 0.6 else 'white' 
                     for x in recent_alerts['risk_score']],
                    [self.colors['danger'] if x else 'white' 
                     for x in recent_alerts['is_predicted_fraud']]
                ],
                align='left'
            )
        )])
        
        fig.update_layout(
            title="Real-time Fraud Alert Feed",
            height=600
        )
        
        return fig
    
    def create_geographic_heatmap(self) -> go.Figure:
        """Create geographic fraud distribution heatmap."""
        # Aggregate by country
        country_stats = self.df.groupby('country').agg({
            'is_actual_fraud': ['sum', 'mean'],
            'amount': 'sum',
            'transaction_id': 'count'
        }).round(2)
        
        country_stats.columns = ['fraud_count', 'fraud_rate', 
                                'total_amount', 'transaction_count']
        country_stats = country_stats.reset_index()
        
        # Create choropleth
        fig = px.choropleth(
            country_stats,
            locations='country',
            locationmode='ISO-3',
            color='fraud_rate',
            hover_data=['fraud_count', 'transaction_count', 'total_amount'],
            color_continuous_scale='Reds',
            title='Fraud Rate by Country'
        )
        
        fig.update_layout(
            geo=dict(
                showframe=False,
                showcoastlines=True,
                projection_type='natural earth'
            ),
            height=500
        )
        
        return fig
    
    def create_model_performance(self) -> go.Figure:
        """Create model performance metrics for data scientists."""
        # Calculate confusion matrix
        tp = ((self.df['is_predicted_fraud'] == True) & 
              (self.df['is_actual_fraud'] == True)).sum()
        fp = ((self.df['is_predicted_fraud'] == True) & 
              (self.df['is_actual_fraud'] == False)).sum()
        tn = ((self.df['is_predicted_fraud'] == False) & 
              (self.df['is_actual_fraud'] == False)).sum()
        fn = ((self.df['is_predicted_fraud'] == False) & 
              (self.df['is_actual_fraud'] == True)).sum()
        
        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Confusion Matrix', 'ROC Curve', 
                          'Precision-Recall', 'Feature Importance'),
            specs=[[{'type': 'heatmap'}, {'type': 'scatter'}],
                   [{'type': 'scatter'}, {'type': 'bar'}]]
        )
        
        # Confusion matrix
        cm_data = [[tn, fp], [fn, tp]]
        fig.add_trace(
            go.Heatmap(
                z=cm_data,
                x=['Predicted Normal', 'Predicted Fraud'],
                y=['Actual Normal', 'Actual Fraud'],
                text=cm_data,
                texttemplate="%{text}",
                colorscale='Blues'
            ),
            row=1, col=1
        )
        
        # ROC Curve (simulated)
        fpr = np.linspace(0, 1, 100)
        tpr = np.sqrt(fpr) * 0.9 + fpr * 0.1  # Simulated good ROC
        fig.add_trace(
            go.Scatter(
                x=fpr, y=tpr,
                mode='lines',
                name='ROC Curve',
                line=dict(color=self.colors['primary'], width=2)
            ),
            row=1, col=2
        )
        fig.add_trace(
            go.Scatter(
                x=[0, 1], y=[0, 1],
                mode='lines',
                name='Random',
                line=dict(color='gray', width=1, dash='dash')
            ),
            row=1, col=2
        )
        
        # Precision-Recall curve (simulated)
        recall = np.linspace(0, 1, 100)
        precision = 0.9 - 0.7 * recall + 0.1 * np.random.normal(0, 0.02, 100)
        fig.add_trace(
            go.Scatter(
                x=recall, y=precision,
                mode='lines',
                name='PR Curve',
                line=dict(color=self.colors['success'], width=2)
            ),
            row=2, col=1
        )
        
        # Feature importance (simulated)
        features = ['V1', 'V2', 'Amount', 'V3', 'V4', 'Time', 'V5', 'V6']
        importance = [0.15, 0.12, 0.11, 0.10, 0.09, 0.08, 0.07, 0.06]
        fig.add_trace(
            go.Bar(
                x=features,
                y=importance,
                marker_color=self.colors['info']
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            title="Model Performance Dashboard",
            height=800,
            showlegend=False
        )
        
        return fig

# Create visualizer and generate dashboards
visualizer = DashboardVisualizer(transactions_df)

# Generate visualizations
print("📈 Generating Dashboard Visualizations...\n")

# Executive KPIs
kpi_fig = visualizer.create_kpi_cards()
kpi_fig.show()

# Real-time feed
feed_fig = visualizer.create_real_time_feed()
feed_fig.show()

# Model performance
perf_fig = visualizer.create_model_performance()
perf_fig.show()

## 5. Implementing Business Impact Analysis

Let's create components that translate technical metrics into business value.

In [ ]:
class BusinessImpactCalculator:
    """Calculate and visualize business impact of fraud detection."""
    
    def __init__(self, transactions_df: pd.DataFrame):
        self.df = transactions_df
        
        # Business parameters
        self.params = {
            'investigation_cost': 25,  # $ per investigation
            'fraud_loss_multiplier': 2.5,  # Total loss is 2.5x transaction amount
            'customer_lifetime_value': 5000,  # Average CLV
            'false_positive_impact': 0.02,  # 2% chance of losing customer
            'operational_cost_per_transaction': 0.10  # $ per transaction processed
        }
    
    def calculate_roi(self) -> Dict:
        """Calculate ROI of fraud detection system."""
        # True positives (fraud caught)
        tp = self.df[
            (self.df['is_predicted_fraud'] == True) & 
            (self.df['is_actual_fraud'] == True)
        ]
        
        # False positives (good transactions blocked)
        fp = self.df[
            (self.df['is_predicted_fraud'] == True) & 
            (self.df['is_actual_fraud'] == False)
        ]
        
        # False negatives (fraud missed)
        fn = self.df[
            (self.df['is_predicted_fraud'] == False) & 
            (self.df['is_actual_fraud'] == True)
        ]
        
        # Calculate financials
        fraud_prevented = tp['amount'].sum() * self.params['fraud_loss_multiplier']
        fraud_losses = fn['amount'].sum() * self.params['fraud_loss_multiplier']
        investigation_costs = len(tp) * self.params['investigation_cost']
        false_positive_costs = (
            len(fp) * self.params['investigation_cost'] + 
            len(fp) * self.params['false_positive_impact'] * 
            self.params['customer_lifetime_value']
        )
        operational_costs = (
            len(self.df) * self.params['operational_cost_per_transaction']
        )
        
        total_benefit = fraud_prevented
        total_cost = (
            investigation_costs + false_positive_costs + 
            operational_costs + fraud_losses
        )
        
        roi = (total_benefit - total_cost) / total_cost * 100
        
        return {
            'fraud_prevented': fraud_prevented,
            'fraud_losses': fraud_losses,
            'investigation_costs': investigation_costs,
            'false_positive_costs': false_positive_costs,
            'operational_costs': operational_costs,
            'total_benefit': total_benefit,
            'total_cost': total_cost,
            'net_benefit': total_benefit - total_cost,
            'roi_percentage': roi
        }
    
    def create_roi_dashboard(self) -> go.Figure:
        """Create ROI visualization dashboard."""
        roi_data = self.calculate_roi()
        
        # Create subplots
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Cost-Benefit Analysis', 'ROI Trend', 
                          'Cost Breakdown', 'Benefit Sources'),
            specs=[[{'type': 'bar'}, {'type': 'scatter'}],
                   [{'type': 'pie'}, {'type': 'waterfall'}]]
        )
        
        # Cost-Benefit comparison
        categories = ['Benefits', 'Costs']
        values = [roi_data['total_benefit'], roi_data['total_cost']]
        fig.add_trace(
            go.Bar(
                x=categories,
                y=values,
                text=[f'${v:,.0f}' for v in values],
                textposition='outside',
                marker_color=['green', 'red']
            ),
            row=1, col=1
        )
        
        # ROI trend (simulated over time)
        days = pd.date_range(end=datetime.now(), periods=30, freq='D')
        roi_trend = 150 + np.cumsum(np.random.normal(2, 10, 30))
        fig.add_trace(
            go.Scatter(
                x=days,
                y=roi_trend,
                mode='lines+markers',
                name='ROI %',
                line=dict(color='blue', width=2)
            ),
            row=1, col=2
        )
        
        # Cost breakdown
        cost_labels = ['Investigation', 'False Positives', 'Operations', 'Fraud Losses']
        cost_values = [
            roi_data['investigation_costs'],
            roi_data['false_positive_costs'],
            roi_data['operational_costs'],
            roi_data['fraud_losses']
        ]
        fig.add_trace(
            go.Pie(
                labels=cost_labels,
                values=cost_values,
                hole=0.4
            ),
            row=2, col=1
        )
        
        # Waterfall chart for net benefit
        waterfall_x = ['Fraud Prevented', 'Investigation Costs', 
                      'False Positive Costs', 'Operational Costs', 
                      'Fraud Losses', 'Net Benefit']
        waterfall_y = [
            roi_data['fraud_prevented'],
            -roi_data['investigation_costs'],
            -roi_data['false_positive_costs'],
            -roi_data['operational_costs'],
            -roi_data['fraud_losses'],
            None  # Will be calculated automatically
        ]
        
        fig.add_trace(
            go.Waterfall(
                x=waterfall_x,
                y=waterfall_y,
                text=[f'${abs(v):,.0f}' if v else '' for v in waterfall_y],
                textposition="outside",
                connector={"line": {"color": "rgb(63, 63, 63)"}}
            ),
            row=2, col=2
        )
        
        fig.update_layout(
            title=f"Business Impact Dashboard - ROI: {roi_data['roi_percentage']:.1f}%",
            height=800,
            showlegend=False
        )
        
        return fig

# Calculate business impact
impact_calculator = BusinessImpactCalculator(transactions_df)
roi_data = impact_calculator.calculate_roi()

print("💰 Business Impact Analysis:\n")
print(f"Fraud Prevented: ${roi_data['fraud_prevented']:,.2f}")
print(f"Total Costs: ${roi_data['total_cost']:,.2f}")
print(f"Net Benefit: ${roi_data['net_benefit']:,.2f}")
print(f"ROI: {roi_data['roi_percentage']:.1f}%\n")

# Create ROI dashboard
roi_fig = impact_calculator.create_roi_dashboard()
roi_fig.show()

## 6. Building the Complete Dashboard Application

Let's create a complete dashboard structure that can be deployed.

In [ ]:
class FraudDashboardApp:
    """
    Complete fraud detection dashboard application.
    
    In production, this would be implemented using Streamlit, Dash, or similar.
    """
    
    def __init__(self):
        self.simulator = TransactionSimulator()
        self.current_user = None
        self.refresh_data()
    
    def refresh_data(self):
        """Refresh transaction data."""
        self.transactions_df = self.simulator.generate_batch(1000)
        self.visualizer = DashboardVisualizer(self.transactions_df)
        self.impact_calculator = BusinessImpactCalculator(self.transactions_df)
    
    def render_sidebar(self) -> Dict:
        """Render sidebar configuration."""
        return {
            'title': '🛡️ FraudGuard Pro',
            'user_selector': {
                'label': 'Select User Persona',
                'options': ['Executive', 'Manager', 'Analyst', 'Data Scientist']
            },
            'filters': {
                'date_range': {'start': datetime.now() - timedelta(days=7), 
                              'end': datetime.now()},
                'risk_threshold': 0.5,
                'model_version': 'All'
            },
            'actions': [
                {'label': 'Refresh Data', 'icon': '🔄'},
                {'label': 'Export Report', 'icon': '📊'},
                {'label': 'Settings', 'icon': '⚙️'}
            ]
        }
    
    def render_main_content(self, user_type: str) -> Dict:
        """Render main content based on user type."""
        content_map = {
            'Executive': {
                'components': [
                    {'type': 'kpi_cards', 'width': 12},
                    {'type': 'geographic_map', 'width': 8},
                    {'type': 'trend_chart', 'width': 4},
                    {'type': 'roi_summary', 'width': 12}
                ]
            },
            'Manager': {
                'components': [
                    {'type': 'team_performance', 'width': 6},
                    {'type': 'workload_distribution', 'width': 6},
                    {'type': 'cost_analysis', 'width': 12},
                    {'type': 'alert_statistics', 'width': 12}
                ]
            },
            'Analyst': {
                'components': [
                    {'type': 'alert_feed', 'width': 12},
                    {'type': 'investigation_tools', 'width': 8},
                    {'type': 'quick_actions', 'width': 4},
                    {'type': 'pattern_analysis', 'width': 12}
                ]
            },
            'Data Scientist': {
                'components': [
                    {'type': 'model_performance', 'width': 12},
                    {'type': 'feature_analysis', 'width': 6},
                    {'type': 'drift_detection', 'width': 6},
                    {'type': 'experiment_results', 'width': 12}
                ]
            }
        }
        
        return content_map.get(user_type, content_map['Executive'])
    
    def get_dashboard_config(self) -> Dict:
        """Get complete dashboard configuration."""
        return {
            'app_name': 'FraudGuard Pro',
            'version': '2.0',
            'theme': {
                'primary_color': '#1f77b4',
                'background_color': '#f0f2f6',
                'text_color': '#262730'
            },
            'layout': {
                'sidebar_width': 300,
                'main_padding': 20,
                'component_spacing': 10
            },
            'features': {
                'real_time_updates': True,
                'export_enabled': True,
                'dark_mode': True,
                'mobile_responsive': True
            },
            'api_endpoints': {
                'data': '/api/v1/transactions',
                'predictions': '/api/v1/predict',
                'metrics': '/api/v1/metrics',
                'export': '/api/v1/export'
            }
        }
    
    def generate_sample_code(self) -> str:
        """Generate sample Streamlit code for the dashboard."""
        code = '''
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime, timedelta

# Page configuration
st.set_page_config(
    page_title="FraudGuard Pro",
    page_icon="🛡️",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #1f77b4;
        text-align: center;
        padding: 2rem;
    }
    .metric-card {
        background-color: #f0f2f6;
        padding: 1rem;
        border-radius: 0.5rem;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
</style>
""", unsafe_allow_html=True)

# Header
st.markdown('<h1 class="main-header">🛡️ FraudGuard Pro</h1>', 
            unsafe_allow_html=True)

# Sidebar
with st.sidebar:
    st.header("Configuration")
    
    # User persona selector
    user_type = st.selectbox(
        "Select User Persona",
        ["Executive", "Manager", "Analyst", "Data Scientist"]
    )
    
    # Date range filter
    date_range = st.date_input(
        "Date Range",
        value=(datetime.now() - timedelta(days=7), datetime.now()),
        max_value=datetime.now()
    )
    
    # Risk threshold
    risk_threshold = st.slider(
        "Risk Threshold",
        min_value=0.0,
        max_value=1.0,
        value=0.5,
        step=0.05
    )
    
    # Refresh button
    if st.button("🔄 Refresh Data"):
        st.experimental_rerun()

# Main content based on user type
if user_type == "Executive":
    # KPI Cards
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        st.metric(
            label="Total Transactions",
            value="10,234",
            delta="+5.2%"
        )
    
    with col2:
        st.metric(
            label="Fraud Rate",
            value="2.3%",
            delta="-0.5%",
            delta_color="inverse"
        )
    
    with col3:
        st.metric(
            label="Detection Accuracy",
            value="94.5%",
            delta="+2.1%"
        )
    
    with col4:
        st.metric(
            label="Fraud Prevented",
            value="$1.2M",
            delta="+12.3%"
        )
    
    # Additional dashboard components would go here...

elif user_type == "Analyst":
    # Real-time alert feed
    st.header("🚨 Real-time Fraud Alerts")
    
    # Alert table would go here...
    
# Footer
st.markdown("---")
st.markdown("Dashboard last updated: " + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        '''
        
        return code

# Create dashboard app
dashboard_app = FraudDashboardApp()

# Display configuration
print("🎯 Dashboard Application Configuration:\n")
config = dashboard_app.get_dashboard_config()
for section, values in config.items():
    print(f"{section.upper()}:")
    if isinstance(values, dict):
        for key, value in values.items():
            print(f"  {key}: {value}")
    else:
        print(f"  {values}")
    print()

# Generate sample code
print("\n📝 Sample Streamlit Implementation:")
print("=" * 50)
sample_code = dashboard_app.generate_sample_code()
print(sample_code[:1000] + "\n... (truncated)")

## 7. Advanced Features: Model Monitoring Dashboard

Let's create a specialized dashboard for monitoring model performance and drift.

In [ ]:
class ModelMonitoringDashboard:
    """Advanced model monitoring and drift detection dashboard."""
    
    def __init__(self, transactions_df: pd.DataFrame):
        self.df = transactions_df
        self.reference_stats = self._calculate_reference_stats()
    
    def _calculate_reference_stats(self) -> Dict:
        """Calculate reference statistics for drift detection."""
        return {
            'fraud_rate': 0.05,  # Expected 5% fraud rate
            'avg_amount': 150.0,
            'risk_score_dist': {
                'mean': 0.3,
                'std': 0.2
            },
            'processing_time': {
                'p50': 15.0,
                'p95': 40.0
            }
        }
    
    def detect_drift(self) -> Dict:
        """Detect various types of model drift."""
        current_fraud_rate = self.df['is_actual_fraud'].mean()
        current_avg_amount = self.df['amount'].mean()
        current_risk_mean = self.df['risk_score'].mean()
        current_risk_std = self.df['risk_score'].std()
        
        drift_metrics = {
            'concept_drift': {
                'detected': abs(current_fraud_rate - self.reference_stats['fraud_rate']) > 0.02,
                'severity': 'high' if abs(current_fraud_rate - self.reference_stats['fraud_rate']) > 0.03 else 'low',
                'current_value': current_fraud_rate,
                'reference_value': self.reference_stats['fraud_rate']
            },
            'data_drift': {
                'amount_drift': {
                    'detected': abs(current_avg_amount - self.reference_stats['avg_amount']) / self.reference_stats['avg_amount'] > 0.2,
                    'current_value': current_avg_amount,
                    'reference_value': self.reference_stats['avg_amount']
                },
                'distribution_drift': {
                    'detected': abs(current_risk_std - self.reference_stats['risk_score_dist']['std']) > 0.1,
                    'current_std': current_risk_std,
                    'reference_std': self.reference_stats['risk_score_dist']['std']
                }
            },
            'performance_drift': {
                'accuracy_drop': self._check_performance_drift(),
                'latency_increase': self._check_latency_drift()
            }
        }
        
        return drift_metrics
    
    def _check_performance_drift(self) -> Dict:
        """Check for performance degradation."""
        # Calculate rolling accuracy
        self.df['correct_prediction'] = (
            self.df['is_predicted_fraud'] == self.df['is_actual_fraud']
        )
        
        # Recent vs historical accuracy
        recent_accuracy = self.df.tail(100)['correct_prediction'].mean()
        historical_accuracy = self.df.head(900)['correct_prediction'].mean()
        
        return {
            'detected': recent_accuracy < historical_accuracy - 0.05,
            'recent_accuracy': recent_accuracy,
            'historical_accuracy': historical_accuracy
        }
    
    def _check_latency_drift(self) -> Dict:
        """Check for processing latency increases."""
        recent_p95 = self.df.tail(100)['processing_time_ms'].quantile(0.95)
        
        return {
            'detected': recent_p95 > self.reference_stats['processing_time']['p95'] * 1.5,
            'current_p95': recent_p95,
            'threshold': self.reference_stats['processing_time']['p95'] * 1.5
        }
    
    def create_drift_dashboard(self) -> go.Figure:
        """Create comprehensive drift monitoring dashboard."""
        drift_metrics = self.detect_drift()
        
        # Create subplots
        fig = make_subplots(
            rows=3, cols=2,
            subplot_titles=(
                'Fraud Rate Drift', 'Risk Score Distribution',
                'Model Accuracy Trend', 'Processing Latency',
                'Feature Drift Heatmap', 'Alert Summary'
            ),
            specs=[
                [{'type': 'scatter'}, {'type': 'histogram'}],
                [{'type': 'scatter'}, {'type': 'box'}],
                [{'type': 'heatmap'}, {'type': 'indicator'}]
            ],
            row_heights=[0.3, 0.3, 0.4]
        )
        
        # Fraud rate over time
        time_groups = pd.cut(self.df.index, bins=20)
        fraud_rate_trend = self.df.groupby(time_groups)['is_actual_fraud'].mean()
        
        fig.add_trace(
            go.Scatter(
                x=list(range(len(fraud_rate_trend))),
                y=fraud_rate_trend.values,
                mode='lines+markers',
                name='Fraud Rate',
                line=dict(color='red', width=2)
            ),
            row=1, col=1
        )
        
        # Reference line
        fig.add_hline(
            y=self.reference_stats['fraud_rate'],
            line_dash="dash",
            line_color="gray",
            annotation_text="Reference",
            row=1, col=1
        )
        
        # Risk score distribution
        fig.add_trace(
            go.Histogram(
                x=self.df['risk_score'],
                nbinsx=30,
                name='Current',
                opacity=0.7
            ),
            row=1, col=2
        )
        
        # Model accuracy trend
        accuracy_trend = self.df.groupby(time_groups)['correct_prediction'].mean()
        
        fig.add_trace(
            go.Scatter(
                x=list(range(len(accuracy_trend))),
                y=accuracy_trend.values,
                mode='lines+markers',
                name='Accuracy',
                line=dict(color='green', width=2)
            ),
            row=2, col=1
        )
        
        # Processing latency boxplot
        fig.add_trace(
            go.Box(
                y=self.df['processing_time_ms'],
                name='Latency',
                boxpoints='outliers'
            ),
            row=2, col=2
        )
        
        # Feature drift heatmap (simulated)
        features = ['V1', 'V2', 'V3', 'V4', 'Amount', 'Time']
        time_periods = ['Week 1', 'Week 2', 'Week 3', 'Week 4']
        drift_matrix = np.random.rand(len(features), len(time_periods)) * 0.3
        
        fig.add_trace(
            go.Heatmap(
                z=drift_matrix,
                x=time_periods,
                y=features,
                colorscale='RdYlBu_r',
                text=np.round(drift_matrix, 2),
                texttemplate='%{text}',
                textfont={"size": 10}
            ),
            row=3, col=1
        )
        
        # Alert summary
        drift_count = sum([
            drift_metrics['concept_drift']['detected'],
            drift_metrics['data_drift']['amount_drift']['detected'],
            drift_metrics['performance_drift']['accuracy_drop']['detected']
        ])
        
        fig.add_trace(
            go.Indicator(
                mode="number+gauge",
                value=drift_count,
                title={'text': "Active Drift Alerts"},
                gauge={
                    'axis': {'range': [0, 5]},
                    'bar': {'color': 'darkred' if drift_count > 2 else 'orange' if drift_count > 0 else 'green'},
                    'steps': [
                        {'range': [0, 1], 'color': 'lightgreen'},
                        {'range': [1, 3], 'color': 'lightyellow'},
                        {'range': [3, 5], 'color': 'lightcoral'}
                    ],
                    'threshold': {
                        'line': {'color': "red", 'width': 4},
                        'thickness': 0.75,
                        'value': 3
                    }
                }
            ),
            row=3, col=2
        )
        
        fig.update_layout(
            title="Model Monitoring & Drift Detection Dashboard",
            height=1000,
            showlegend=False
        )
        
        return fig

# Create monitoring dashboard
monitor = ModelMonitoringDashboard(transactions_df)
drift_metrics = monitor.detect_drift()

print("🔍 Drift Detection Results:\n")
print(f"Concept Drift: {'DETECTED' if drift_metrics['concept_drift']['detected'] else 'OK'}")
print(f"  Current fraud rate: {drift_metrics['concept_drift']['current_value']:.2%}")
print(f"  Reference rate: {drift_metrics['concept_drift']['reference_value']:.2%}")
print()
print(f"Data Drift (Amount): {'DETECTED' if drift_metrics['data_drift']['amount_drift']['detected'] else 'OK'}")
print(f"Performance Drift: {'DETECTED' if drift_metrics['performance_drift']['accuracy_drop']['detected'] else 'OK'}")

# Create drift dashboard
drift_fig = monitor.create_drift_dashboard()
drift_fig.show()

## 8. Dashboard Best Practices and Deployment

Let's summarize best practices for production dashboard deployment.

In [ ]:
# Dashboard best practices
best_practices = {
    "Design Principles": [
        "Know your audience - design for specific user personas",
        "Progressive disclosure - show summary first, details on demand",
        "Visual hierarchy - most important metrics prominently displayed",
        "Consistent color coding - red for alerts, green for good",
        "Mobile-first design - dashboards must work on all devices"
    ],
    
    "Performance Optimization": [
        "Implement data caching for expensive queries",
        "Use pagination for large datasets",
        "Lazy loading for complex visualizations",
        "WebSocket connections for real-time updates",
        "CDN for static assets"
    ],
    
    "Security Considerations": [
        "Role-based access control (RBAC)",
        "Data masking for sensitive information",
        "Audit logging for all dashboard actions",
        "Secure API endpoints with authentication",
        "Regular security audits"
    ],
    
    "Deployment Architecture": [
        "Container-based deployment (Docker/Kubernetes)",
        "Load balancing for high availability",
        "Database read replicas for analytics queries",
        "Message queuing for asynchronous updates",
        "Monitoring and alerting infrastructure"
    ],
    
    "User Experience": [
        "Intuitive navigation with breadcrumbs",
        "Contextual help and tooltips",
        "Export functionality for all visualizations",
        "Customizable dashboard layouts",
        "Dark mode support"
    ]
}

print("📋 Dashboard Best Practices:\n")
for category, practices in best_practices.items():
    print(f"\n{category}:")
    for practice in practices:
        print(f"  ✓ {practice}")

# Deployment checklist
deployment_checklist = """
\n🚀 Production Dashboard Deployment Checklist:

1. Pre-deployment:
   □ User acceptance testing completed
   □ Performance benchmarks met
   □ Security review passed
   □ Documentation updated
   □ Training materials prepared

2. Infrastructure:
   □ Production environment configured
   □ SSL certificates installed
   □ Backup strategy implemented
   □ Monitoring tools connected
   □ CDN configured

3. Deployment:
   □ Blue-green deployment setup
   □ Database migrations completed
   □ Feature flags configured
   □ Rollback plan tested
   □ Health checks passing

4. Post-deployment:
   □ User onboarding completed
   □ Performance monitoring active
   □ Feedback collection enabled
   □ Analytics tracking verified
   □ Support team briefed
"""

print(deployment_checklist)

## Exercise: Build Your Own Dashboard Components

Try implementing these additional dashboard features:

1. **A/B Testing Dashboard**
   - Compare model versions side-by-side
   - Statistical significance testing
   - Winner determination

2. **Alert Management System**
   - Alert prioritization
   - Assignment workflow
   - Investigation timeline

3. **Customer Impact Analysis**
   - False positive impact on customers
   - Customer satisfaction metrics
   - Complaint tracking

4. **Compliance Dashboard**
   - Regulatory reporting
   - Audit trail visualization
   - SLA compliance tracking

In [ ]:
# Exercise template - A/B Testing Dashboard

class ABTestingDashboard:
    """Build an A/B testing dashboard for model comparison."""
    
    def __init__(self, control_data: pd.DataFrame, treatment_data: pd.DataFrame):
        self.control = control_data
        self.treatment = treatment_data
    
    def calculate_test_statistics(self) -> Dict:
        """
        Calculate A/B test statistics.
        
        TODO: Implement statistical tests
        - Calculate conversion rates
        - Perform significance testing
        - Calculate confidence intervals
        - Determine sample size adequacy
        """
        pass
    
    def create_comparison_dashboard(self) -> go.Figure:
        """
        Create visual comparison of A/B test results.
        
        TODO: Create visualizations
        - Side-by-side metrics
        - Confidence interval plots
        - Time series comparison
        - Winner probability gauge
        """
        pass
    
    def generate_recommendation(self) -> str:
        """
        Generate recommendation based on test results.
        
        TODO: Implement decision logic
        - Check statistical significance
        - Consider business impact
        - Account for sample size
        - Provide clear recommendation
        """
        pass

print("🧪 A/B Testing Dashboard template created!")
print("\nImplement the TODO methods to build your own A/B testing dashboard!")

## Summary and Key Takeaways

In this comprehensive tutorial, we've built a professional fraud detection dashboard system with:

### 🎯 Key Components Implemented

1. **Multi-Persona Dashboard Architecture**
   - Executive KPI dashboards
   - Analyst investigation tools
   - Manager team dashboards
   - Data scientist model monitoring

2. **Advanced Visualizations**
   - Real-time alert feeds
   - Geographic heatmaps
   - Interactive charts
   - Business impact metrics

3. **Business Intelligence Features**
   - ROI calculations
   - Cost-benefit analysis
   - Impact assessment
   - Trend analysis

4. **Model Monitoring**
   - Drift detection
   - Performance tracking
   - Feature analysis
   - Alert systems

### 💡 Best Practices Learned

- **User-Centric Design**: Always design for specific user needs
- **Progressive Disclosure**: Show summaries first, details on demand
- **Visual Hierarchy**: Most important information most prominent
- **Real-time Updates**: Keep data fresh and relevant
- **Mobile Responsiveness**: Ensure accessibility on all devices

### 🚀 Next Steps

1. **Deploy to Production**
   - Choose framework (Streamlit, Dash, custom)
   - Set up infrastructure
   - Implement security

2. **Add Advanced Features**
   - Machine learning insights
   - Predictive analytics
   - Automated reporting

3. **Integrate with Systems**
   - Connect to real data sources
   - Implement webhooks
   - Add notification systems

4. **Gather Feedback**
   - User testing sessions
   - Analytics tracking
   - Iterative improvements

You now have the knowledge to build professional, enterprise-grade ML dashboards that deliver real business value!

Happy dashboarding! 📊🎉